In [1]:
import numpy as np
from scipy.stats import gamma as gamma_dist

In [2]:
lambda_0 = 2
beta = np.array([1, 2, 3])
tau = np.array([0.1, 0.2, 0.3])
kappa = np.array([[0.5, 0.5, 0.5, 0], [0.5, 0, 0.5, 0.5], [0.5, 0, 0.5, 0.5]])
w = np.random.dirichlet([1/3] * 3, 4).T
t_vec = np.array([0.1, 0.5, 0.7, 0.9, 1.2])

In [3]:
def hawkes_likelihood(lambda_0, beta: np.ndarray, tau: np.ndarray, w: np.ndarray,
                      kappa: np.ndarray, t_vec: np.ndarray, n: int):
    """
    hawkes似然函数
    :param w: base kernel的权重矩阵
    :param n: 当前的样本顺序
    :param t_vec: 时间戳向量(观测数据)
    :param kappa: matrix
    :param lambda_0:
    :param beta: array
    :param tau: array
    :return:
    """
    lambda_all = np.zeros(n)
    for j in np.arange(n):  # j range
        kappa_neq_zero_index_j = np.argwhere(kappa[j, :] != 0)[:, 0]
        print(f'kappa {j+1} 非零元素的索引：{kappa_neq_zero_index_j}')
        i = np.arange(j + 1)
        tj_minus_ti = t_vec[j] - t_vec[i]  # t[i]是一个array，因此tj_minus_ti也是一个array
        print(f'tj-ti: {tj_minus_ti}')

        t_div_tau_func = np.vectorize(np.divide, signature='(),(n)->(n)')
        beta_times_tau_func = np.vectorize(lambda x, y: x * y, signature='(n),(n)->(n)')

        gamma = beta_times_tau_func(beta, np.exp(- t_div_tau_func(tj_minus_ti, tau)))
        numerator = (w[:, kappa_neq_zero_index_j].T @ gamma.T).T
        denominator = np.count_nonzero(kappa[: j + 1, :], axis=1).reshape(-1, 1)
        lambda_j = np.einsum('ij->', numerator / denominator)  # 计算lambda_j
        lambda_all[j] = lambda_j  # 保存lambda_j
    # 计算似然函数
    lh = np.exp(-(lambda_0 + np.sum(lambda_all))) * np.prod(lambda_all)
    return lh

In [4]:
hawkes_likelihood(lambda_0=lambda_0, beta=beta, tau=tau, w=w, kappa=kappa, t_vec=t_vec, n=1)

kappa 1 非零元素的索引：[0 1 2]
tj-ti: [0.]


0.04111675457244679

In [5]:
np.sum(kappa, axis=1)

array([1.5, 1.5, 1.5])